In [37]:
import json
import base64
from requests_toolbelt import sessions
from requests_toolbelt.auth.handler import AuthHandler
from requests.adapters import HTTPAdapter
from requests import Session
from requests.packages.urllib3.util.retry import Retry
import types
import json
import logging
import getpass
from collections import OrderedDict

In [4]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

DEFAULT_TIMEOUT = 60 # seconds

class TimeoutHTTPAdapter(HTTPAdapter):
    '''
    
    '''
    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

In [5]:
def auth(session: Session):
    '''
    Authenticates into the RW API
    '''
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}RW api{bcolors.ENDC}')
    
    headers = {'Content-Type': 'application/json'}
    payload = json.dumps({ 'email': f'{input(f"Email: ")}',
                           'password': f'{getpass.getpass(prompt="Password: ")}'})
    response = session.post('/auth/login',  headers = headers,  data = payload)
    
    print(f'{bcolors.OKGREEN}Successfully logged into RW api{bcolors.ENDC}')
    
    return response.json().get('data').get('token')

In [6]:
def rwApiSession(serverUrl: str = "https://api.resourcewatch.org", authenticate:bool =False)-> Session:
    '''
    Creates a sesion object for making calls to the RW api.
    '''
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    
    assert_status_hook = lambda response, *args, **kwargs: response.raise_for_status()

    httpsSession = sessions.BaseUrlSession(base_url = serverUrl )
    
    httpsSession.mount("https://", TimeoutHTTPAdapter(max_retries=retries))
    httpsSession.hooks["response"] = [assert_status_hook]
    
    if authenticate:
        token = auth(httpsSession)
        httpsSession.headers['Authorization'] = f'Bearer {token}'
    
    return httpsSession

In [80]:
def cartoApiSession(account: str, baseUrl: str = "https://{account}.carto.com", apiToken: bool = False)-> Session:
    '''
    '''
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    
    assert_status_hook = lambda response, *args, **kwargs: response.raise_for_status()

    httpsSession = sessions.BaseUrlSession(base_url = baseUrl.format(account=account))
    
    httpsSession.mount("https://", TimeoutHTTPAdapter(max_retries=retries))
    httpsSession.hooks["response"] = [assert_status_hook]
    
    if apiToken:
        token = getpass.getpass(prompt="Token: ")
        httpsSession.auth = (account, token)
    
    return httpsSession
        

In [82]:
rwSession = rwApiSession()
cartoSession = cartoApiSession('wri-rw', apiToken = True)

Token: ········


In [64]:
payload = {
    'q': "SELECT name_0, geostore_prod, cartodb_id FROM gadm36_0"
}
data = cartoSession.get('api/v2/sql', params = payload ).json()
data

{'rows': [{'name_0': 'Afghanistan',
   'geostore_prod': '8077f92339820ecd08c5c228a9af22f9',
   'cartodb_id': 2},
  {'name_0': 'Angola',
   'geostore_prod': 'c0e30c8d35f81d8d19e2c0f5dd7e0798',
   'cartodb_id': 3},
  {'name_0': 'Anguilla',
   'geostore_prod': '33ca8954b5b8cdc268d4c28f7eb74868',
   'cartodb_id': 4},
  {'name_0': 'Åland',
   'geostore_prod': '8dd6597f7ea0173f128e2d31a367bcd2',
   'cartodb_id': 5},
  {'name_0': 'Albania',
   'geostore_prod': '5ae24123b16bcb8ba1a4c377d1e8c237',
   'cartodb_id': 6},
  {'name_0': 'Global',
   'geostore_prod': 'fe855e0073d3a7c45ccd5f653841bb8d',
   'cartodb_id': 257},
  {'name_0': 'Andorra',
   'geostore_prod': 'da7df6772a356f026afadd5ac119c415',
   'cartodb_id': 7},
  {'name_0': 'United Arab Emirates',
   'geostore_prod': 'be0fafa391273cba57809aa1fc7ec6fd',
   'cartodb_id': 8},
  {'name_0': 'Argentina',
   'geostore_prod': '1447c190081d8a22a1ce4b8241978ea3',
   'cartodb_id': 9},
  {'name_0': 'Armenia',
   'geostore_prod': '719cc7ed10f5d844d319

In [30]:
rwSession.get(f"/v1/geostore/{data['rows'][0]['geostore_prod']}").json()

{'data': {'type': 'geoStore',
  'id': '8077f92339820ecd08c5c228a9af22f9',
  'attributes': {'geojson': {'crs': {},
    'type': 'FeatureCollection',
    'features': [{'geometry': {'coordinates': [[[[68.526432037, 31.754346848],
          [68.423027039, 31.755356789],
          [68.320888519, 31.757463455],
          [68.216522217, 31.809566498],
          [68.11668396, 31.758705139],
          [68.053501129, 31.676791191],
          [67.958660126, 31.627516747],
          [67.852203369, 31.603038788],
          [67.767662048, 31.54354763],
          [67.670005798, 31.505005836],
          [67.569564819, 31.534693718],
          [67.59469223, 31.434301376],
          [67.716213226, 31.409849167],
          [67.682884216, 31.306822777],
          [67.57465744, 31.27294445],
          [67.471939087, 31.238893509],
          [67.376014709, 31.194928169],
          [67.276683807, 31.182472229],
          [67.177516937, 31.203194618],
          [67.077178955, 31.204027176],
          [67.03588

In [84]:
try:
    for geostore in data['rows']:
        geostoreObject = rwSession.get(f"/v1/geostore/{geostore['geostore_prod']}").json()
        geojson = OrderedDict(geostoreObject.get('data',{}).get('attributes',{}).get('geojson',{}).get('features')[0])
        
        geojson['properties'] = {"geostore_country_name": geostore['name_0']}
        # we order type properties becouse geostore complains becouse we don't provide it first
        geometry = OrderedDict(geojson['geometry'])
        geometry.move_to_end('type', last=False)
        geojson['geometry'] = geometry
        geojson.move_to_end('type', last=False)
        
        response = rwSession.post("/v1/geostore",
                                  headers = {'Content-Type':'application/json'},
                                  data = json.dumps({'geojson': {"type": "FeatureCollection",
                                                      'features':[geojson]}
                                         })
                                 )
        geostore_id = response.json().get('data',{}).get('id')
        if geostore['geostore_prod'] != geostore_id:
            print(f"geostore needs to update from {geostore['geostore_prod']} to {geostore_id}")
            payload = {
                'q': f"UPDATE gadm36_0 SET geostore_prod ='{geostore_id}' where cartodb_id = {geostore['cartodb_id']}"
                }
            cartoSession.get('api/v2/sql', params = payload).json()
            
        
        print(f"{bcolors.OKGREEN}Successfully update of {geostore['name_0']} with geostore_id {geostore['geostore_prod']} {bcolors.ENDC}")
        
except Exception as e:
    print(e)
    

geostore needs to update from 8077f92339820ecd08c5c228a9af22f9 to 9549f30301a423ab6658aa60c2b6b99d
Successfully update of Afghanistan with geostore_id 8077f92339820ecd08c5c228a9af22f9 
geostore needs to update from c0e30c8d35f81d8d19e2c0f5dd7e0798 to d7f7a24b5db3a16bd4db62db562c2543
Successfully update of Angola with geostore_id c0e30c8d35f81d8d19e2c0f5dd7e0798 
geostore needs to update from 33ca8954b5b8cdc268d4c28f7eb74868 to 43061f08e134e4a8f4e854ef75b366a8
Successfully update of Anguilla with geostore_id 33ca8954b5b8cdc268d4c28f7eb74868 
geostore needs to update from 8dd6597f7ea0173f128e2d31a367bcd2 to a839cb5934b9578771d3399ef86dd80a
Successfully update of Åland with geostore_id 8dd6597f7ea0173f128e2d31a367bcd2 
geostore needs to update from 5ae24123b16bcb8ba1a4c377d1e8c237 to 705323cdcd27a1989af27d1935ad092e
Successfully update of Albania with geostore_id 5ae24123b16bcb8ba1a4c377d1e8c237 
geostore needs to update from fe855e0073d3a7c45ccd5f653841bb8d to 7f67768cc49a3f095824814d7a1

Successfully update of Switzerland with geostore_id 33bc7f5334d2bda483d7a613fe263edf 
geostore needs to update from 67f17c727da2d107f078887ebb0038ff to 965f36617fd50c5dfbc7a11b3e4a5ce0
Successfully update of Chile with geostore_id 67f17c727da2d107f078887ebb0038ff 
geostore needs to update from f4599aaeedacb9e6ddca177a93795e4c to 680d79ab40930f6c4fb09d40e0bb0111
Successfully update of China with geostore_id f4599aaeedacb9e6ddca177a93795e4c 
geostore needs to update from 56ccf2992455b1e344cbc72c252cfa5f to edc51f10c561cce217b6f27905737ff3
Successfully update of Côte d'Ivoire with geostore_id 56ccf2992455b1e344cbc72c252cfa5f 
geostore needs to update from cc6d0585dced222eaeec44b0669c988f to 8cc30cb3b9c96f8bd5ceaf7664b7a752
Successfully update of Cameroon with geostore_id cc6d0585dced222eaeec44b0669c988f 
geostore needs to update from ad228718a734c11e750f6e1c839ac9e6 to 84d6952917b0af9656514530cb2f8501
Successfully update of Democratic Republic of the Congo with geostore_id ad228718a734c11

geostore needs to update from 40e92fa840d5b367034f204c47be4683 to 65b913b895c9b11352775aee1bba95ae
Successfully update of Guadeloupe with geostore_id 40e92fa840d5b367034f204c47be4683 
geostore needs to update from 8b7ffe19b232b49cbff558e19fea0abe to 50b7de492a8cbc0c28a975e566446c37
Successfully update of Gambia with geostore_id 8b7ffe19b232b49cbff558e19fea0abe 
geostore needs to update from f31b6bc7d2d8c599bdf014a40a641c31 to 0f1ae959239a9cd7f1ad89580909ed0e
Successfully update of Guinea-Bissau with geostore_id f31b6bc7d2d8c599bdf014a40a641c31 
geostore needs to update from 2c18c3a4995cdc18615c05fd3d4284d4 to 3e27b9b4734fac1ca39280b84db574c0
Successfully update of Equatorial Guinea with geostore_id 2c18c3a4995cdc18615c05fd3d4284d4 
geostore needs to update from 35cc0fc4233fe8f5d194060913c7b0b2 to 9bb20f669ec67a5645f6449f508b0b7f
Successfully update of Greece with geostore_id 35cc0fc4233fe8f5d194060913c7b0b2 
geostore needs to update from 80249ec341405c760f852de3cdbef8b6 to d8bded61e954

geostore needs to update from 6ad43edbc1c72068de8d0260a5d4455f to bf07bec656827dc1f3e6aea6804c2f11
Successfully update of Saint Lucia with geostore_id 6ad43edbc1c72068de8d0260a5d4455f 
geostore needs to update from 66810efcfbe7affb8163d10cc91c725c to 00c60a2ccacd9fd65b2f3611a644990d
Successfully update of Liechtenstein with geostore_id 66810efcfbe7affb8163d10cc91c725c 
geostore needs to update from 29b32224cfb06d882d945ca3887ac7ad to a2c82e4ac3d884da0bb232220312942c
Successfully update of Sri Lanka with geostore_id 29b32224cfb06d882d945ca3887ac7ad 
geostore needs to update from 3745e4653e2220e70ce33988c7f5a501 to f519316f138c6cc44a522121336f465a
Successfully update of Lesotho with geostore_id 3745e4653e2220e70ce33988c7f5a501 
geostore needs to update from 1542e4ed9c25de8ed6f6c76c1d18a4fd to 00a5f4adef49aaa2ed2df7da2a2f6e9b
Successfully update of Lithuania with geostore_id 1542e4ed9c25de8ed6f6c76c1d18a4fd 
geostore needs to update from e5adfaaaa25214f86cffdfbef254dde9 to 3222bc0aa429bbb

geostore needs to update from b774217e9ce8cd6bba21dd4f87a973a6 to b24155e48494212c5134e743571e10f2
Successfully update of Oman with geostore_id b774217e9ce8cd6bba21dd4f87a973a6 
geostore needs to update from c5278c246ee1800a310b853fdef9ee13 to f08fab4dc1b13ff6ba3488464d8a8b7a
Successfully update of Pakistan with geostore_id c5278c246ee1800a310b853fdef9ee13 
geostore needs to update from 52f79df34fce7dec28b9e8aca32a192f to c14102c7181dda1d10eaa68ce17590c2
Successfully update of Panama with geostore_id 52f79df34fce7dec28b9e8aca32a192f 
geostore needs to update from ee731c66552936ed13d783bfb62b7833 to b36f8863dcd875059a86fc615a6936b7
Successfully update of Pitcairn Islands with geostore_id ee731c66552936ed13d783bfb62b7833 
geostore needs to update from 42313519526eda1a1dd315183a4a1b35 to 0eb11896899330b5478e1723d08fc91a
Successfully update of Peru with geostore_id 42313519526eda1a1dd315183a4a1b35 
geostore needs to update from 3bac85e52de8bbc9574fdc66d0783c82 to 6571959c78e1039abb23e6e46d

geostore needs to update from fa59eec91dd3fe2c2016f0fa22e580e7 to e036fcb1b0ef8a61cddde49038e6268a
Successfully update of Turks and Caicos Islands with geostore_id fa59eec91dd3fe2c2016f0fa22e580e7 
geostore needs to update from 975379f7b751ba549a034dfa7470032b to 76b4c14b73a726b55d9de1f72e6c78c9
Successfully update of Chad with geostore_id 975379f7b751ba549a034dfa7470032b 
geostore needs to update from 4e167cc80461053d49a4565f2e066871 to 2222289a99338cde3e3b0c0958677078
Successfully update of Togo with geostore_id 4e167cc80461053d49a4565f2e066871 
geostore needs to update from c64a6ef0874ba2c7668a37e73822aa0d to 3a6979e82dc19cd08d1958813ffa6a3c
Successfully update of Thailand with geostore_id c64a6ef0874ba2c7668a37e73822aa0d 
geostore needs to update from 7210db415f7716942fdb23eb2e21efc6 to 3db4955cdcadcfb286984733717bce19
Successfully update of Tajikistan with geostore_id 7210db415f7716942fdb23eb2e21efc6 
geostore needs to update from b22112a561ff77c45debe7af4adc121e to bcff2cabb7d229